In [2]:
!pip install faiss-cpu sentence-transformers transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 21.3 MB/s eta 0:00:00


In [4]:
# -*- coding: utf-8 -*-
"""genai-8

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1KPM7ohxj_e4U6IypCeybKShftyjq9mXi
"""


import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import json

# 1. Prepare your documents (structured + unstructured)
structured_docs = [
    {"id": 1, "name": "John Doe", "age": 30, "job": "Engineer"},
    {"id": 2, "name": "Jane Smith", "age": 25, "job": "Data Scientist"},
]

unstructured_docs = [
    "John Doe is an engineer with 10 years of experience.",
    "Jane Smith works as a data scientist and loves machine learning.",
]

# Convert structured docs to text (for retrieval embedding)
def structured_to_text(doc):
    return json.dumps(doc)

all_docs_text = [structured_to_text(d) for d in structured_docs] + unstructured_docs

# 2. Embed documents
embedder = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = embedder.encode(all_docs_text, convert_to_numpy=True)

# 3. Build FAISS index
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)  # L2 distance index
index.add(doc_embeddings)

# 4. Load generation model and tokenizer (e.g. T5-base)
model_name = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
generator = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def retrieve(query, k=3):
    query_embedding = embedder.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, k)
    return [all_docs_text[i] for i in indices[0]]

def generate_answer(query, retrieved_docs):
    # Concatenate query and retrieved docs as context for generation
    context = " ".join(retrieved_docs)
    input_text = f"question: {query} context: {context}"

    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)
    outputs = generator.generate(**inputs, max_length=100)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Example query
query = "Who is Jane Smith and what does she do?"

# Retrieve relevant documents
retrieved = retrieve(query, k=3)
print("Retrieved docs:", retrieved)

# Generate answer based on retrieved docs
answer = generate_answer(query, retrieved)
print("Generated answer:", answer)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Retrieved docs: ['Jane Smith works as a data scientist and loves machine learning.', '{"id": 2, "name": "Jane Smith", "age": 25, "job": "Data Scientist"}', 'John Doe is an engineer with 10 years of experience.']
Generated answer: data scientist
